First import required packages

In [4]:
# install pip -> open a terminal and type 'sudo apt install python3-pip'
# install hv plot -> in the terminal type 'pip install hvplot'

#pip install pandas

import pandas as pd
import requests
import subprocess as sp
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

#pip install ipykernel
#pip install pandas
#pip install jupyter_bokeh
#pip install nbconvert
#https://panel.holoviz.org/api/panel.widgets.html

ModuleNotFoundError: No module named 'pandas'

Next get data from ONS
Get link from: https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/deathsregisteredinenglandandwalesseriesdrreferencetables
xls = pd.ExcelFile('https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/deathsregisteredinenglandandwalesseriesdrreferencetables/2022/dr2022corrected.xlsx')

The url doesn't accept "non-browser" requests. The default header of Python requests is 'User-Agent': 'python-requests/2.13.0', need to pass different headers as an argument
https://stackoverflow.com/questions/59297759/python-requests-module-to-download-an-excel-file-from-a-download-link-which-po

In [40]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}
response = requests.get('https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/deathsregisteredinenglandandwalesseriesdrreferencetables/2022/dr2022corrected.xlsx', headers=headers, verify=False)
with open("DeathsData.xlsx", 'wb') as f:
    f.write(response.content)

# Bring in a tab of the xlsx as a data frame
dfpub = pd.read_excel('DeathsData.xlsx', sheet_name='2a', skiprows=4)
print(dfpub)

c:\ProgramData\miniforge3\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ons.gov.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


     Area code                    Area of usual residence     Geography level  \
0    K04000001               ENGLAND, WALES AND ELSEWHERE             Country   
1    E92000001                                    ENGLAND             Country   
2    E12000001                                 NORTH EAST              Region   
3    E06000047                              County Durham   Unitary Authority   
4    E06000005                                 Darlington   Unitary Authority   
..         ...                                        ...                 ...   
377  W11000030  Cwm Taf Morgannwg University Health Board  Local Health Board   
378  W11000025          Hywel Dda University Health Board  Local Health Board   
379  W11000024                Powys Teaching Health Board  Local Health Board   
380  W11000031        Swansea Bay University Health Board  Local Health Board   
381  J99000001  Usual residence outside England and Wales                 [z]   

     Persons\n(Deaths) Pers

Next get data from the Lake - note this will only work if you are on the network and have read access to the table and views in the query. Changes to the query can be performed by editing 'GetLakeDeaths.SQL'. Specify the path to your batch file and run the batch file using subprocess.

In [19]:
import subprocess as sp
batch_file_path = 'GetLakeDeaths.bat'
sp.call([batch_file_path])

0

Read in the Lake data

In [35]:
# Read the text file into a DataFrame
dflake = pd.read_csv('outputfile.txt', sep='\s+')

# Display the DataFrame
print(dflake)

       Dashboard  Period                                            GeoCode  \
0    -----------  ------  ----------------------------------------------...   
1              1    2018                                                NaN   
2              1    2018                                                NaN   
3              1    2018                                                NaN   
4              1    2018                                                NaN   
..           ...     ...                                                ...   
572            4    2017                                                NaN   
573            4    2017                                                NaN   
574            4    2018                                                NaN   
575            4    2018                                                NaN   
576         (575    rows                                          affected)   

                                               GeoN

Join the datasets together

In [41]:
# Merge the two dataframes
result = pd.merge(dflake, dfpub, left_on='GeoCode', right_on='Area code', how='left')

print(result)

       Dashboard  Period                                            GeoCode  \
0    -----------  ------  ----------------------------------------------...   
1              1    2018                                                NaN   
2              1    2018                                                NaN   
3              1    2018                                                NaN   
4              1    2018                                                NaN   
..           ...     ...                                                ...   
572            4    2017                                                NaN   
573            4    2017                                                NaN   
574            4    2018                                                NaN   
575            4    2018                                                NaN   
576         (575    rows                                          affected)   

                                               GeoN

In [42]:
variables = result.columns.tolist()

print(variables)


['Dashboard', 'Period', 'GeoCode', 'GeoName', 'VariableType', 'VariableA', 'VariableB', 'Value', 'Area code', 'Area of usual residence', 'Geography level', 'Persons\n(Deaths)', 'Persons\n(ASMR)', 'Persons\n(Lower CL)', 'Persons\n(Upper CL)', 'Males\n(Deaths)', 'Males\n(ASMR)', 'Males\n(Lower CL)', 'Males\n(Upper CL)', 'Females\n(Deaths)', 'Females\n(ASMR)', 'Females\n(Lower CL)', 'Females\n(Upper CL)']


In [50]:
# Create a sample DataFrame
data = {'A': [1, 2, 3, 4, 5], 'B': ['a', 'b', 'c', 'd', 'e']}
df = pd.DataFrame(result)

# Apply a filter to select rows where values in column 'A' are greater than 2
filtered_df = df[df['Dashboard'] == '3']

# Print the filtered DataFrame
print(filtered_df)

    Dashboard Period    GeoCode GeoName VariableType VariableA VariableB  \
247         3   2018        NaN     NaN           LA       NaN       NaN   
248         3   2018  E06000001     NaN           LA       NaN       NaN   
249         3   2018  E06000002     NaN           LA       NaN       NaN   
250         3   2018  E06000003     NaN           LA       NaN       NaN   
251         3   2018  E06000004     NaN           LA       NaN       NaN   
..        ...    ...        ...     ...          ...       ...       ...   
557         3   2018  E09000029     NaN           LA       NaN       NaN   
558         3   2018  E09000030     NaN           LA       NaN       NaN   
559         3   2018  E09000031     NaN           LA       NaN       NaN   
560         3   2018  E09000032     NaN           LA       NaN       NaN   
561         3   2018  E09000033     NaN           LA       NaN       NaN   

       Value  Area code Area of usual residence  ... Persons\n(Lower CL)  \
247  35730.

In [53]:
print(filtered_df[['GeoCode', 'Value', 'Persons\n(Deaths)']])


       GeoCode    Value  Persons\n(Deaths)
247        NaN  35730.0                NaN
248  E06000001   1014.0             1064.0
249  E06000002   1454.0             1535.0
250  E06000003   1594.0             1738.0
251  E06000004   1972.0             2037.0
..         ...      ...                ...
557  E09000029   1586.0             1548.0
558  E09000030   1132.0             1140.0
559  E09000031   1419.0             1529.0
560  E09000032   1465.0             1514.0
561  E09000033   1075.0             1050.0

[315 rows x 3 columns]
